In [1]:
import numpy as np
import pandas as pd
from tensorflow.python.lib.io import file_io
from skimage.transform import resize
from keras.models import load_model
from keras.utils.np_utils import to_categorical

In [2]:
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [3]:
#make it "Inception" for Inception-v3
train_model =  "ResNet" 
if train_model == "Inception":
    img_width, img_height =139, 139
elif train_model == "ResNet":
    img_width, img_height =197, 197

In [4]:
test_dataset='/Users/anushkapatil/Desktop/thesis/testing_dataset.csv'

In [5]:
model = load_model("/Users/anushkapatil/Desktop/thesis/trainedmodels/ResNet-50.h5")

In [6]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 197, 197, 3) 0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 99, 99, 64)   9408        input_1[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 99, 99, 64)   256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 99, 99, 64)   0           conv1/7x7_s2/bn[0][0]            
____________________________________________________________________________________________

In [7]:
batch_size = 1

In [8]:
def preprocess_input(x):
    if train_model == "Inception":
        x /= 127.5
        x -= 1.
        return x
    elif train_model == "ResNet":
        x -= 128.8006	# np.mean(train_dataset)
        x /= 64.6497	# np.std(train_dataset)
        return x
    elif train_model == "miniXception":
        x = x.astype('float32')
        x = x/255.0
        x = (x - 0.5)*2.0
        return x

In [9]:
def get_data(dataset):
    file_stream = file_io.FileIO(dataset, mode='r')
    data = pd.read_csv(file_stream)
    pixels = data['pixels'].tolist()
    images = np.empty((len(data), img_height, img_width, 3))
    i = 0

    for pixel_sequence in pixels:
        single_image = [float(pixel) for pixel in pixel_sequence.split(' ')]  
        single_image = np.asarray(single_image).reshape(48, 48) 
        single_image = resize(single_image, (img_height, img_width), order = 3, mode = 'constant') 
        ret = np.empty((img_height, img_width, 3))  
        ret[:, :, 0] = single_image
        ret[:, :, 1] = single_image
        ret[:, :, 2] = single_image
        images[i, :, :, :] = ret
        i += 1
    
    images = preprocess_input(images)
    labels = to_categorical(data['emotion'])

    return images, labels    

In [10]:
images, labels = get_data(test_dataset)

In [11]:
predictions = model.predict(images,batch_size= batch_size)

predicted_classes= np.argmax(predictions, axis = 1)
#true_classes =  labels
true_classes=np.argmax(labels,axis=1)
class_names = list(["Anger", "Disgust", "Fear", "Happinness", "Sadness", "Surprise", "Neutral"])

In [12]:
accuracy = accuracy_score(true_classes, predicted_classes, normalize = True)

In [13]:
report = classification_report(true_classes, predicted_classes,target_names = class_names)


In [14]:
print(train_model)
print("Accuracy:")
print(accuracy)
print("\n")
print("Report:")
print(report)


ResNet
Accuracy:
0.7124547227640011


Report:
              precision    recall  f1-score   support

       Anger       0.64      0.64      0.64       491
     Disgust       0.73      0.65      0.69        55
        Fear       0.58      0.49      0.53       528
  Happinness       0.88      0.91      0.89       879
     Sadness       0.58      0.60      0.59       594
    Surprise       0.81      0.80      0.80       416
     Neutral       0.68      0.74      0.71       626

    accuracy                           0.71      3589
   macro avg       0.70      0.69      0.69      3589
weighted avg       0.71      0.71      0.71      3589

